In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
test_data=pd.read_excel('Test_Set.xlsx')

In [3]:
test_data.head()

,Customer_ID,Store_Code
0,1800000006365760,60298
1,1800000006365760,60299
2,1800000006365760,60300
3,1800000006365760,60301
4,1800000006365820,60298


In [4]:
test_data['Store_Code'].value_counts()

60298    100000
60299    100000
60300    100000
60301    100000
Name: Store_Code, dtype: int64

In [5]:
customer_trans=pd.read_excel('Customer_Transaction.xlsx')

In [6]:
customer_trans.count()

Territory           544649
Business            544649
Year                544649
Week                544649
Store_Code          544649
City_Name           505464
Store_Type          505464
Transaction_Type    544649
Return_Reason        37014
Customer_ID         544649
Invoices            544649
Item_Count          544649
Revenue             544649
Discount            544649
Units_Sold          544649
dtype: int64

In [7]:
customer_trans.columns 

Index(['Territory', 'Business', 'Year', 'Week', 'Store_Code', 'City_Name',
       'Store_Type', 'Transaction_Type', 'Return_Reason', 'Customer_ID',
       'Invoices', 'Item_Count', 'Revenue', 'Discount', 'Units_Sold'],
      dtype='object')

In [8]:
t_count=customer_trans[['Customer_ID','Store_Code','Year']].groupby(['Customer_ID','Store_Code'])['Year'].agg({"Transaction Count":len}).reset_index()

C:\Users\dwiver2\AppData\Local\Programs\Python\Python35\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [9]:
customer_str_trns=pd.pivot_table(t_count,index='Customer_ID',columns='Store_Code',values='Transaction Count')

In [10]:
customer_str_trns.fillna(0,inplace=True)

In [11]:

for x in customer_str_trns.columns :
    customer_str_trns[x]=customer_str_trns[x].apply(lambda x : 1 if x>1 else 0)

In [12]:
customer_demo=pd.read_excel('Customer_Demographics.xlsx')

In [13]:
customer_demo.head()

,Customer_ID,Territory,Nationality,Income_Range,Job_Type,Marital_Status,Gender,State,Language,Loyalty_Status,Birth_date,Age,Points,First_txn_dt,Last_accr_txn_dt,Last_rdm_txn_dt
0,1800000006365760,United Arab Emirates,INDIA,Below 5000,Services,Married,M,Abu Dhabi,English,Gold,01JUN1959:00:00:00,59.0,814.0,02NOV2010:00:00:00,27SEP2018:21:45:25,11MAR2018:19:11:57
1,1800000006365820,United Arab Emirates,UNITED ARAB EMIRATES,Below 5000,Services,Married,F,Abu Dhabi,Arabic,Silver,01JAN1900:00:00:00,NaN,212.0,06DEC2010:00:00:00,16AUG2016:21:08:28,16MAR2015:13:39:12
2,1800000006366060,United Arab Emirates,SRI LANKA,Below 5000,Services,Married,M,Abu Dhabi,English,Silver,07JUN1959:00:00:00,59.0,186.0,17FEB2011:00:00:00,31AUG2018:20:52:18,30JAN2017:19:19:19
3,1800000006366230,United Arab Emirates,JORDAN,Below 5000,Services,Married,F,Abu Dhabi,Arabic,Silver,21JAN1962:00:00:00,57.0,126.0,30OCT2010:00:00:00,30JUL2018:20:30:05,20JUN2017:17:57:27
4,1800000006366570,United Arab Emirates,OCCUPIED PALESTINIAN TERRITORY,Below 5000,Services,Married,M,Abu Dhabi,Arabic,Silver,01JAN1900:00:00:00,NaN,197.0,10NOV2010:00:00:00,03SEP2018:22:10:56,17JUN2017:23:08:28


In [14]:
#customer_demo['Income_Range'].value_counts()
customer_demo['Job_Type'].value_counts()

Unspecified                                       92896
Others                                             4337
Services                                           2409
Housewife/Homemaker                                  83
Student                                              70
Manager                                              65
Officer                                              50
Engineer                                             30
Self Employed                                        22
Accountant                                           14
Sales Executive                                       6
PRESIDENT/SECY/TREASURER TO CLUB/ASSOC/SOCIETY        4
Supervisor                                            3
Director                                              3
Doctor                                                2
Technician                                            2
Administartor                                         2
Clerk                                           

In [15]:
from datetime import datetime

In [16]:
customer_demo['lt_accr_date']=customer_demo.Last_accr_txn_dt.apply(lambda x:None if ((x=='nan') or (x is np.nan) or (x==np.nan)) else datetime.strptime(x,"%d%b%Y:%H:%M:%S"))

In [17]:
customer_demo['lt_accr_date'][customer_demo['lt_accr_date'].isnull()==True]

38830   NaT
96273   NaT
96850   NaT
97268   NaT
97459   NaT
99339   NaT
Name: lt_accr_date, dtype: datetime64[ns]

In [18]:
customer_demo['lt_rdm_date']=customer_demo.Last_rdm_txn_dt.apply(lambda x:None if ((x=='nan') or (x is np.nan) or (x==np.nan)) else datetime.strptime(x,"%d%b%Y:%H:%M:%S"))
customer_demo['ft_txn_date']=customer_demo.First_txn_dt.apply(lambda x:None if ((x=='nan') or (x is np.nan) or (x==np.nan)) else datetime.strptime(x,"%d%b%Y:%H:%M:%S"))

In [19]:

customer_demo['ds_ft_purchase']=(customer_demo['lt_accr_date']-customer_demo['ft_txn_date'])
customer_demo['lt_date']=customer_demo[['lt_accr_date','lt_rdm_date']].apply(max,axis=1)

In [20]:
customer_demo['lt_month']=customer_demo['lt_date'].apply(lambda x:x.month)
customer_demo['lt_year']=customer_demo['lt_date'].apply(lambda x:x.year)

In [21]:
store_master=pd.read_excel('Store_Master.xlsx')

In [22]:
store_master.head()

,Territory,Business,Region,Region_Code,Store_Code,Store_Name,Mall_Name,Store_Format,Sales_Per_Day,Store_Launch_Date,Train_Test_Store,Geo_Field,Store_Size_Sq_Ft,Customer_Count,Total_Revenue
0,United Arab Emirates,Max,MX - Abu Dhabi,602,60077,Max Zayed City Mall,City Mall - Abu Dhabi-Uae,NaN,544.074849,2011-11-02,Train,"25.618167, 53.441646",4852.220341,22237.0,1.131676e+07
1,United Arab Emirates,Max,MX - Sharjah,606,60073,Max Safeer Mall,Safeer Mall Sharjah-Uae,NaN,1251.656549,2009-08-21,Train,"25.309997, 55.378550",13789.949661,88372.0,2.603446e+07
2,United Arab Emirates,Max,MX - Dubai,601,60065,Max Gardens,Ibn Battuta Mall-Uae,NaN,3319.885702,2005-04-15,Train,"25.044684, 55.120275",15609.687991,188836.0,6.905362e+07
3,United Arab Emirates,Max,MX - Dubai,601,60219,MX -Dubai Festival City,Dubai Festival City-Uae,Stand Alone Stores,1211.477261,2014-06-12,Train,"25.218486, 55.359313",6845.497058,96733.0,2.519873e+07
4,United Arab Emirates,Max,MX - Ras Al Khaimah,605,60071,Max Rak,STANDALONE,NaN,2221.872874,2006-03-20,Train,NaN,12385.047461,70456.0,4.621496e+07


In [23]:
store_master[(store_master['Region_Code'].isin([602,606])) & (store_master['Train_Test_Store']=='Train')]

#store_master[(store_master.Region_Code==602)&(store_master.Train_Test_Store!='Test')]


,Territory,Business,Region,Region_Code,Store_Code,Store_Name,Mall_Name,Store_Format,Sales_Per_Day,Store_Launch_Date,Train_Test_Store,Geo_Field,Store_Size_Sq_Ft,Customer_Count,Total_Revenue
0,United Arab Emirates,Max,MX - Abu Dhabi,602,60077,Max Zayed City Mall,City Mall - Abu Dhabi-Uae,NaN,544.074849,2011-11-02,Train,"25.618167, 53.441646",4852.220341,22237.0,1.131676e+07
1,United Arab Emirates,Max,MX - Sharjah,606,60073,Max Safeer Mall,Safeer Mall Sharjah-Uae,NaN,1251.656549,2009-08-21,Train,"25.309997, 55.378550",13789.949661,88372.0,2.603446e+07
6,United Arab Emirates,Max,MX - Sharjah,606,60075,Max Sharjah Qassimia,STANDALONE,NaN,2268.223683,2011-02-11,Train,NaN,9582.354867,113795.0,4.717905e+07
9,United Arab Emirates,Max,MX - Abu Dhabi,602,60067,Max Marina Mall,Abudhabi Marina Mall-Uae,NaN,1699.560982,2011-12-14,Train,"24.476186, 54.321439",8673.534535,89351.0,3.535087e+07
14,United Arab Emirates,Max,MX - Abu Dhabi,602,60058,Max Bawabat Al Sharq,Bawabat Al Sharq Malluae-Uae,NaN,1965.523053,2011-12-08,Train,"24.312255, 54.619651",8792.292763,80204.0,4.088288e+07
15,United Arab Emirates,Max,MX - Sharjah,606,60074,Max Sharjah,STANDALONE,NaN,2631.139338,2004-07-21,Train,NaN,3467.799125,137253.0,5.472770e+07
17,United Arab Emirates,Max,MX - Sharjah,606,60076,Max Shj City Centre,Sharjah City Centre-Uae,NaN,2265.965914,2012-03-23,Train,"25.325119, 55.393559",93125.240189,165012.0,4.713209e+07
18,United Arab Emirates,Max,MX - Abu Dhabi,602,60069,Max Mushrif Mall,Mushrif Mall Abudhabi-Uae,NaN,1813.094697,2011-07-30,Train,"24.434721, 54.412599",6698.117964,114246.0,3.771237e+07
19,United Arab Emirates,Max,MX - Abu Dhabi,602,60079,Max Abudhabi Deerfields,Deerfields Mall-Uae,Stand Alone Stores,1193.119529,2013-11-23,Train,"24.321044, 54.480138",3699.470014,54832.0,2.481689e+07
23,United Arab Emirates,Max,MX - Abu Dhabi,602,60233,MX -Yas Mall,Yas Mall-Uae,Mall Stores,1045.779749,2014-11-25,Train,"24.322808, 54.480221",16455.386430,75505.0,2.175222e+07


In [24]:
from itertools import combinations_with_replacement,combinations
#import geopy.distance

In [40]:
store_master['Geo_Field'].isnull().sum()

8

In [51]:
store_master['Geo_Field'].fillna("",inplace=True)

In [55]:
store_master['Source']=store_master['Geo_Field'].apply(lambda x : x.split(',')[0])

In [59]:
store_master['Destination']=store_master['Geo_Field'].apply(lambda x : None if x=="" else x.split(',')[1])

In [64]:
store_master['Destination'].dropna()

0      53.441646
1      55.378550
2      55.120275
3      55.359313
8      55.397985
9      54.321439
10     55.286850
11     55.243562
12     55.241679
13     55.925038
14     54.619651
16     56.302260
17     55.393559
18     54.412599
19     54.480138
21     54.480221
23     54.480221
24     55.407763
25     55.198099
26     55.808654
27     55.330340
29     54.523955
30     54.382945
31     54.372679
32     55.489202
33     54.357529
34     55.781629
Name: Destination, dtype: object